# Agenda

1. The iterator protocol
    - Adding iteration to your classes
    - Different techniques for that
2. Generator functions
3. Generator comprehensions
4. Decorators
5. Threading and multiprocessing 

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
        
p1 = Person('name1')        
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1!
Hello, name2!


In [2]:
import weakref

In [3]:
p3 = weakref.ref(p2)


In [4]:
p3

<weakref at 0x10c289310; to 'Person' at 0x10c2874c0>

In [5]:
p3.ref

AttributeError: 'weakref' object has no attribute 'ref'

In [7]:
w = weakref.ref(Person('name10'))

In [8]:
w

<weakref at 0x10b2aabd0; dead>

In [9]:
type(w)

weakref

In [10]:
dir(w)

['__call__',
 '__callback__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [11]:
p3 = weakref.ref(p2)


In [13]:
p3

<weakref at 0x10c289310; to 'Person' at 0x10c2874c0>

In [14]:
p1 = Person('name1')
p2 = Person('name2')

all_people = {1: p1, 2: p2}

In [15]:
del(p1)

In [16]:
len(all_people)

2

In [17]:
p1 = Person('name1')
p2 = Person('name2')

all_people = weakref.WeakValueDictionary()

all_people[1] = p1
all_people[2] = p2

In [18]:
all_people

In [20]:
list(all_people.items())

[(1, <__main__.Person at 0x10c263760>), (2, <__main__.Person at 0x10c2636a0>)]

In [21]:
len(all_people)

2

In [22]:
del(p1)

In [23]:
len(all_people)

2

In [24]:
len(all_people)

2

In [25]:
list(all_people.items())

[(1, <__main__.Person at 0x10c263760>), (2, <__main__.Person at 0x10c2636a0>)]

In [30]:
p1 = Person('name1')
p2 = Person('name2')

all_people = weakref.WeakValueDictionary()

all_people[1] = p1
all_people[2] = p2

In [31]:
del(p1)

In [32]:
len(all_people)

1

In [33]:
del(p2)

In [34]:
len(all_people)

0